In [1]:
import statistics

import pandas as pd
import numpy as np
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score
import warnings
from sklearn.ensemble import RandomForestClassifier

warnings.filterwarnings('ignore')

In [2]:
class Config:
    RANDOM_SEED = 13
    NUM_FOLDS = 5
    TARGET_COL_NAME = "song_popularity"
    CATEGORICAL_COLS = ["audio_mode", "time_signature", "key"]

DATA_PATH = "./data/"

In [3]:
df_train = pd.read_csv(DATA_PATH + "train.csv")
df_test = pd.read_csv(DATA_PATH + "test.csv")

In [4]:
# split the training dataframe into kfolds for cross validation. We do this before any processing is done
# on the data. We use stratified kfold if the target distribution is unbalanced
def strat_kfold_dataframe(df, target_col_name, num_folds=5):
    # we create a new column called kfold and fill it with -1
    df["kfold"] = -1
    # randomize of shuffle the rows of dataframe before splitting is done
    df = df.sample(frac=1, random_state=Config.RANDOM_SEED).reset_index(drop=True)
    # get the target data
    y = df[target_col_name].values
    skf = model_selection.StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)
    for fold, (train_index, val_index) in enumerate(skf.split(X=df, y=y)):
        df.loc[val_index, "kfold"] = fold    
    return df     

df_train = strat_kfold_dataframe(df_train, target_col_name=Config.TARGET_COL_NAME)
df_train.head()

,id,song_duration_ms,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,audio_mode,speechiness,tempo,time_signature,audio_valence,song_popularity,kfold
0,37020,241605.0,0.043564,0.668579,0.940644,0.001371,5.0,0.660077,-5.334750,1,0.098402,145.488755,3,0.752138,0,4
1,5531,NaN,0.136844,0.857481,0.780121,0.003005,5.0,0.200062,-12.289020,0,0.040943,122.209006,3,0.747739,1,4
2,15625,181809.0,0.176350,0.722215,0.904362,0.000554,8.0,0.094481,-4.299832,0,0.061723,119.554958,4,0.648478,0,2
3,10671,167442.0,NaN,0.354930,0.496588,0.000845,8.0,0.112777,-13.174818,0,0.042100,106.928615,3,0.531688,1,2
4,4393,164342.0,0.112951,0.633773,0.922505,0.004053,5.0,0.141997,-5.169450,0,0.036036,126.757199,4,0.661839,0,1


In [5]:
cont_cols = ['song_duration_ms', 'acousticness', 'danceability', 'energy', 
            'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'audio_valence']

In [6]:
def impute_df_col(df, col_name, imputer):
    imputed_col = imputer.fit_transform(df[col_name].to_numpy().reshape(-1, 1))
    return pd.Series(imputed_col.reshape(-1))    

In [7]:
def impute_missing_values(df, cols, col_type="cont"):    
    if col_type == "cont":
        imputer = SimpleImputer(missing_values=np.nan, strategy="mean")
    elif col_type == "cat":
        imputer = SimpleImputer(missing_values=np.nan, strategy="most_frequent")            
    for col in cols:
        df[col] = impute_df_col(df, col, imputer)
    return df

df_train = impute_missing_values(df_train, Config.CATEGORICAL_COLS, col_type="cat")
df_train = impute_missing_values(df_train, cont_cols, col_type="cont")
df_test = impute_missing_values(df_test, Config.CATEGORICAL_COLS, col_type="cat")
df_test = impute_missing_values(df_test, cont_cols, col_type="cont")

In [8]:
def add_missing_col(df, cols_with_nulls):
    for col_name in cols_with_nulls:        
        df[col_name + "_missing"] = [int(item) for item in df[col_name].isna().values]
    return df        

train_cols_withnulls = [col for col in df_train.columns if df_train[col].isnull().any()]
test_cols_withnulls = [col for col in df_test.columns if df_test[col].isnull().any()]
df_train = add_missing_col(df_train, train_cols_withnulls)
df_test = add_missing_col(df_test, test_cols_withnulls)

In [9]:
[col for col in df_train.columns if df_train[col].isnull().any()]

[]

In [10]:
[col for col in df_test.columns if df_test[col].isnull().any()]

[]

In [11]:
def col_one_hot_encode(df, cols):    
    one_hot_enc = OneHotEncoder(handle_unknown="ignore", sparse=False)
    one_hot_enc.fit(df[cols])
    return one_hot_enc.transform(df[cols])

In [19]:
def get_input_features(df):
    non_cont_cols = Config.CATEGORICAL_COLS + ["id", "kfold", "song_popularity_proba", Config.TARGET_COL_NAME]
    cont_col_names = [item for item in df.columns.values.tolist() if item not in non_cont_cols]     
    X_cont = df[cont_col_names].to_numpy()       
    X_cat_one_hot = col_one_hot_encode(df, Config.CATEGORICAL_COLS)    
    X = np.concatenate((X_cont, X_cat_one_hot), axis=1)    
    return X

def get_fold_data(fold, df):
    df_train = df[df.kfold != fold]
    df_val = df[df.kfold == fold]
    X_train = get_input_features(df_train)
    y_train = df_train[Config.TARGET_COL_NAME].to_numpy()
    X_val = get_input_features(df_val)
    y_val = df_val[Config.TARGET_COL_NAME].to_numpy()
    return X_train, y_train, X_val, y_val

In [13]:
def run_training(model, train_X, train_y, val_X, val_y):        
    scaler = StandardScaler()
    train_X_scaled = scaler.fit_transform(train_X)    
    val_X_scaled = scaler.fit_transform(val_X)    
    model.fit(train_X_scaled, train_y.ravel())
    val_y_pred_proba = model.predict_proba(val_X_scaled)
    auc = roc_auc_score(val_y, val_y_pred_proba[:, 1], average="weighted")
    return auc, model, val_y_pred_proba[:, 1]

In [14]:
import optuna

def rf_objective(trial):       
    params = {        
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
        "max_depth": trial.suggest_int("max_depth", 4, 20),
        "min_samples_leaf": trial.suggest_categorical("min_samples_leaf", [1, 2, 4]),
        "min_samples_split": trial.suggest_categorical("min_samples_split", [2, 4, 8]),
        "max_features": trial.suggest_categorical("max_features", ["auto", "log2"])
    }
    rf_model = RandomForestClassifier(
                n_estimators=params["n_estimators"],                 
                max_depth=params["max_depth"],
                min_samples_leaf=params["min_samples_leaf"],
                min_samples_split=params["min_samples_split"],
                max_features=params["max_features"],
                random_state=Config.RANDOM_SEED,
                n_jobs=-1
            )   

    fold_auc = []
    for fold in range(Config.NUM_FOLDS):
        train_X, train_y, val_X, val_y = get_fold_data(fold, df_train)
        auc_score, _, _ = run_training(rf_model, train_X, train_y, val_X, val_y)
        fold_auc.append(auc_score)
    mean_auc = statistics.mean(fold_auc)                
    return mean_auc

study = optuna.create_study(direction="maximize", study_name="RFModelTuning")    
study.optimize(rf_objective, n_trials=20)
print("Best trial:")
print(study.best_params)

[I 2022-01-28 18:28:23,682] A new study created in memory with name: RFModelTuning
[I 2022-01-28 18:28:37,402] Trial 0 finished with value: 0.5682960497746609 and parameters: {'n_estimators': 155, 'max_depth': 12, 'min_samples_leaf': 1, 'min_samples_split': 8, 'max_features': 'log2'}. Best is trial 0 with value: 0.5682960497746609.
[I 2022-01-28 18:29:12,767] Trial 1 finished with value: 0.5664979216554858 and parameters: {'n_estimators': 751, 'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 4, 'max_features': 'log2'}. Best is trial 0 with value: 0.5682960497746609.
[I 2022-01-28 18:29:43,111] Trial 2 finished with value: 0.571596434633612 and parameters: {'n_estimators': 703, 'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 4, 'max_features': 'log2'}. Best is trial 2 with value: 0.571596434633612.
[I 2022-01-28 18:30:18,230] Trial 3 finished with value: 0.5714305311218595 and parameters: {'n_estimators': 708, 'max_depth': 13, 'min_samples_leaf': 4, 'min_samples_

Best trial:
{'n_estimators': 703, 'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 4, 'max_features': 'log2'}


In [16]:
# Best trial:
# {'n_estimators': 703, 'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 4, 'max_features': 'log2'}

In [17]:
#rf_model_params = {"n_estimators": 600, "random_state": 42, "max_depth": 8}
rf_model_params = study.best_params
rf_model = RandomForestClassifier(
                n_estimators=rf_model_params["n_estimators"],                 
                max_depth=rf_model_params["max_depth"],
                min_samples_leaf=rf_model_params["min_samples_leaf"],
                min_samples_split=rf_model_params["min_samples_split"],
                max_features=rf_model_params["max_features"],
                random_state=Config.RANDOM_SEED,
                n_jobs=-1
            )     

In [20]:
fold_metrics_model = []
test_preds = {}
df_train["song_popularity_proba"] = 0.0
for fold in range(Config.NUM_FOLDS):
    X_train, y_train, X_val, y_val = get_fold_data(fold, df_train)
    fold_auc_score, model, fold_val_preds = run_training(rf_model, X_train, y_train, X_val, y_val)
    print(f"fold {fold } auc score = {fold_auc_score}")
    # add the validation probability predictions for the fold to a new column in train data
    df_train[df_train.kfold == fold]["song_popularity_proba"] = fold_val_preds    
    X_test = get_input_features(df_test)
    scaler = StandardScaler()
    X_test_scaled = scaler.fit_transform(X_test)
    fold_test_preds = model.predict_proba(X_test_scaled)[:, 1]
    pred_col_name = f"fold_{fold}_test_preds"
    test_preds[pred_col_name] = fold_test_preds    
    fold_metrics_model.append((round(fold_auc_score, 4), model))

fold 0 auc score = 0.5717816670630163
fold 1 auc score = 0.5778660203639332
fold 2 auc score = 0.5711537144697939
fold 3 auc score = 0.567197775045496
fold 4 auc score = 0.5699829962258205


In [21]:
fold_metrics = [item[0] for item in fold_metrics_model]
print(f"auc scores = {fold_metrics}")    
cv_auc_mean = statistics.mean(fold_metrics)
cv_auc_stdev = statistics.stdev(fold_metrics)
print(f"mean auc across folds = {cv_auc_mean}, auc stdev across folds = {cv_auc_stdev}")

auc scores = [0.5718, 0.5779, 0.5712, 0.5672, 0.57]
mean auc across folds = 0.57162, auc stdev across folds = 0.003930903204099517


In [22]:
df_test_preds = pd.DataFrame(test_preds)
test_pred_cols = [f"fold_{fold}_test_preds" for fold in range(Config.NUM_FOLDS)]
df_test_preds["mean_test_pred"] = df_test_preds[test_pred_cols].mean(axis=1)
print(f"Completed prediction for {len(df_test)} test rows")
df_submission = pd.read_csv(DATA_PATH + 'sample_submission.csv')
df_submission['song_popularity']= df_test_preds["mean_test_pred"]
df_submission.to_csv('submission_rf.csv',index=False)
df_submission.head()

Completed prediction for 10000 test rows


,id,song_popularity
0,0,0.369985
1,1,0.420580
2,2,0.336024
3,3,0.347541
4,4,0.365547


In [23]:
df_submission[df_submission.song_popularity > 0.5]

,id,song_popularity
1003,1003,0.512736
1558,1558,0.501997
2344,2344,0.514235
2958,2958,0.528304
4028,4028,0.500179
4161,4161,0.505733
4629,4629,0.511304
5146,5146,0.543439
5777,5777,0.508473
6204,6204,0.514941


In [24]:
rf_val_preds = df_train[["id", "song_popularity_proba", "song_popularity"]]
rf_val_preds.to_csv("rf_val_preds.csv")
print("Saved validation predictions for all folds to csv")

Saved validation predictions for all folds to csv
